In [4]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd

# data preparation

### assigning a percentage of the gap area to a tree ID  

In [5]:
NORTH_PATH = './data/correlation_data/mgr - do analizy/North'
SOUTH_PATH = './data/correlation_data/mgr - do analizy/South'

In [6]:
def folder_names(path):
    
    folders = []
    folders = [f for f in sorted(os.listdir(path))]
    
    return folders

In [7]:
def names_of_files_in_directory(path):

    files=[]
    files = [f for f in sorted(os.listdir(path)) if f.endswith('.shp')]
   
    return files

In [8]:
def number_of_erase_shapefiles(files):
    count = 0

    for element in files:
        list_x = element.split('_')
        if len(list_x) == 4:
            count += 1
            
    return count

In [9]:
def create_result_df(path):

    for folder_name in folder_names(path):
        year = str(folder_name.split('_')[-2])
        files = names_of_files_in_directory(path + '/' + folder_name)
        number_of_files = number_of_erase_shapefiles(files)
        export_path = path[:-5] + '/Results/' + path[-5:] +  '/' + year + '.csv'

        data = {}

        for i in range(number_of_files):
            file_path = path + '/' + folder_name + '/' + 'martwe_drzewa_' + year + '_Erase' + str(i) + '_CLIP_' + year + '.shp'
            df = gpd.read_file(file_path)
            return df
            break
            
            data[i] = (df['area_x'].sum() / 157.08) * 100

        export_df = pd.DataFrame(data.items(), columns=['ID', 'area'])
        export_df.to_csv(path_or_buf = export_path)


In [10]:
# create_result_df(NORTH_PATH)
# create_result_df(SOUTH_PATH)

### assigning the diameter at breast height to a tree based on the tree number and ID

In [11]:
PATH2 = './data/correlation_data/mgr - do analizy/Spatial joins results'
PATH3 = './data/correlation_data/mgr - do analizy/Results/North'
PATH4 = './data/correlation_data/mgr - do analizy/Results/South'

In [12]:
dbh_dead = pd.read_excel(r'./data/Martwe pnie od 2014.xls')
dbh_dead = dbh_dead[dbh_dead[' pi'].astype(str).str.isdigit()]
dbh_dead.rename(columns = {' pi':'Pierśnica'}, inplace = True)
dbh_dead.head()

,numer,kwa,Unnamed: 2,ka,Pierśnica,Rok,Uwagi,kategorie:,1 - stojące
5,6002.0,80,NaN,2,65,2014,NaN,NaN,NaN
6,6004.0,87.88,NaN,2,50,2014,NaN,NaN,NaN
7,6006.0,87.88,NaN,2,39,2014,NaN,NaN,NaN
8,6008.0,101.102,NaN,2,49,2014,NaN,NaN,NaN
9,6011.0,94.95,NaN,1,36,2014,NaN,NaN,NaN


In [79]:
def assign_stand_gaps(df, year):

    north_path = PATH3 + '/' + str(year) + '.csv'
    gap_file = pd.read_csv(north_path)
    gap_file.rename(columns = {'area': 'north_gap'}, inplace = True)

    df = df.merge(gap_file, left_on = 'ORIG_FID', right_on = 'ID')

    south_path = PATH4 + '/' + str(year) + '.csv'
    gap_file = pd.read_csv(south_path)
    gap_file.rename(columns = {'area': 'south_gap'}, inplace = True)

    df = df.merge(gap_file, left_on = 'ORIG_FID', right_on = 'ID')

    return df

In [94]:
def assign_dbh_to_number(year, assign_dbh: bool = False):
    
    file_path = '{}/{}.shp'.format(PATH2, year)
    df = gpd.read_file(file_path)
    df.rename(columns = {'areasol_19': 'AreaSol_19', 'piersnica': 'Pierśnica'}, inplace = True)
    
    if assign_dbh:
        try:
            df = df.merge(dbh_dead, left_on = 'Numer', right_on = 'numer')
        except:
            df = df.merge(dbh_dead, left_on = 'numer', right_on = 'numer')
            
    df = assign_stand_gaps(df, year)

    if year == 2014: df['NEAR_DIST'] = 0    
    df = df[['layer_name', 'AreaSol_19', 'NEAR_DIST' ,'Pierśnica', 'north_gap', 'south_gap']]
    df["Pierśnica"] = pd.to_numeric(df["Pierśnica"])
    df.replace(0, np.nan, inplace=True)
        
    return df

In [108]:
years_2014_2022 = [
    assign_dbh_to_number(2014, True),
    assign_dbh_to_number(2015, True),
    assign_dbh_to_number(2016, True),
    assign_dbh_to_number(2017, True),
    assign_dbh_to_number(2018),
    assign_dbh_to_number(2019),
    assign_dbh_to_number(2020),
    #assign_dbh_to_number(2021),
    assign_dbh_to_number(2022)]

years_2014_2018 = [
    assign_dbh_to_number(2014, True),
    assign_dbh_to_number(2015, True),
    assign_dbh_to_number(2016, True),
    assign_dbh_to_number(2017, True),
    assign_dbh_to_number(2018)]

years_2019_2022 = [
    assign_dbh_to_number(2019),
    assign_dbh_to_number(2020),
    #assign_dbh_to_number(2021),
    assign_dbh_to_number(2022)]

df_2014_2022 = pd.concat(years_2014_2022, axis = 0)
df_2014_2018 = pd.concat(years_2014_2018, axis = 0)
df_2019_2022 = pd.concat(years_2014_2022, axis = 0)

In [109]:
df_2014_2022.iloc[:,1:]

,AreaSol_19,NEAR_DIST,Pierśnica,north_gap,south_gap
0,1.211900e+06,NaN,39.0,55.837026,56.033422
1,1.148300e+06,NaN,50.0,50.086962,54.023682
2,1.054120e+06,NaN,65.0,50.086962,54.023682
3,1.087380e+06,NaN,49.0,50.086962,54.023682
4,8.597100e+05,NaN,57.0,50.086962,54.023682
...,...,...,...,...,...
29,1.215210e+06,28.201735,47.0,38.568946,8.878463
30,1.174409e+06,10.829162,36.0,55.131971,6.583461
31,1.176715e+06,27.577149,44.0,65.281385,11.009677
32,1.137940e+06,29.170122,35.0,66.766616,11.191495


In [110]:
df_2014_2022.iloc[:,1:].corr()

,AreaSol_19,NEAR_DIST,Pierśnica,north_gap,south_gap
AreaSol_19,1.000000,0.080363,-0.122345,-0.004179,0.055696
NEAR_DIST,0.080363,1.000000,-0.039988,-0.094003,-0.165482
Pierśnica,-0.122345,-0.039988,1.000000,0.045012,-0.022015
north_gap,-0.004179,-0.094003,0.045012,1.000000,0.262508
south_gap,0.055696,-0.165482,-0.022015,0.262508,1.000000


In [111]:
df_2014_2018.iloc[:,1:].corr()

,AreaSol_19,NEAR_DIST,Pierśnica,north_gap,south_gap
AreaSol_19,1.000000,0.168606,-0.133761,0.003666,0.049513
NEAR_DIST,0.168606,1.000000,-0.053746,-0.107772,-0.190260
Pierśnica,-0.133761,-0.053746,1.000000,0.039578,-0.056659
north_gap,0.003666,-0.107772,0.039578,1.000000,0.268620
south_gap,0.049513,-0.190260,-0.056659,0.268620,1.000000


In [112]:
df_2019_2022.iloc[:,1:].corr()

,AreaSol_19,NEAR_DIST,Pierśnica,north_gap,south_gap
AreaSol_19,1.000000,0.080363,-0.122345,-0.004179,0.055696
NEAR_DIST,0.080363,1.000000,-0.039988,-0.094003,-0.165482
Pierśnica,-0.122345,-0.039988,1.000000,0.045012,-0.022015
north_gap,-0.004179,-0.094003,0.045012,1.000000,0.262508
south_gap,0.055696,-0.165482,-0.022015,0.262508,1.000000
